# Tutorial 5: Langchain Agent

In this tutorial we explore how to combine LangChain's modular interfaces with a robust C++ runtime, and the portability benefits of GenC.

## Initial Setup

Before proceeding, please follow the instructions in
[Tutorial 1](https://github.com/google/generative_computing/tree/master/generative_computing/docs/tutorials/tutorial_1_simple_cascade.ipynb)
to set up your environment, connect Jupyter, and run the command below to run
the GenC imports we're going to use.

In [ ]:
import generative_computing.python as genc
from generative_computing.python import authoring
from generative_computing.python import interop
from generative_computing.python import runtime
from generative_computing.python.examples import executor

Let's also import a few other packages we're going to use:

In [ ]:
import textwrap
import langchain
from langchain.prompts import PromptTemplate

## Define Tools

`WolframAlpha` is an example of a tool that could be used by a variety of
agents. In order to promote its use as a tool within LangChain, you can derive
from the `CustomTool` class we provide with GenC, as shown below. This class,
defined in
[custom_tool.py](https://github.com/google/generative_computing/tree/master/generative_computing/python/interop/langchain/custom_tool.py), inherits from the `LangChain`'s `tools.BaseTool`,
and just makes provisions to include GenC code embedded with it.

Note that the body of `computation` contains the IR you're already familiar
with, we're just wrapping it in a way that facilitates composability with the
external SDK.

In [ ]:
class WolframAlpha(genc.interop.langchain.CustomTool):
  """A specific implementation of CustomTool, including an appid for identification."""

  def __init__(self, appid: str):
    name = "WolframAlpha"
    description = (
        "https://products.wolframalpha.com/api/documentation, one capability is"
        " to evaluate math expression."
    )
    # Extract math equation from a text, then calls WolfraAlpha
    computation = (
        genc.interop.langchain.CustomChain()
        | genc.authoring.create_custom_function("/react/extract_math_question")
        | genc.authoring.create_wolfram_alpha(appid)
        # Template Engine will extract the result from response JSON
        | genc.authoring.create_inja_template(
            "{% if queryresult.pods"
            " %}{{queryresult.pods.0.subpods.0.plaintext}}{% endif %}"
        )
    )

    super().__init__(
        name=name, description=description, computation=computation
    )

## Define Agent with Tool

Now, let's define the agent proper.

Similarly to what we did with the tool, here we use a GenC-supplied
`CustomAgent` class that inherits from its LangChain base class counterpart,
and makes provisions to include GenC code, such as the use of a GenC-defined
model. This is defined in
[custom_agent.py](https://github.com/google/generative_computing/tree/master/generative_computing/python/interop/langchain/custom_agent.py).

In this example, we already moved the reusable agent/reasoning code into the
LangChain-to-IR converter (see function `create_computation_from_agent`
defined in
[create_computation.py](https://github.com/google/generative_computing/tree/master/generative_computing/python/interop/langchain/create_computation.py),
so you no longer need to write that - it's generatedand bundled-in
automatically for you.

In [ ]:
class MathToolAgent(genc.interop.langchain.CustomAgent):
  """An agent that combines ReAct reasoning loop with WolframAlpha tools."""

  def __init__(self, appid: str, api_key: str):

    system_instruction = """
    Solve a question answering task with interleaving Thought, Action,
    Observation steps.
    Thought can reason about the current situation
    Action can be only two types:
    (1) Math[query], Useful for when you need to solve math problems.
    (2) Finish[answer], which returns the answer as a number terminates.
    Here's an example:
    Question: what is the result of power of 2 + 1?
    Thought: power of 2
    Action: Math[pow(2)]
    Observation: 4
    Thought: I can now process the answer.
    Action: Math[4+1]
    Observation: 5
    Thought: Seems like I got the answer.
    Action: Finish[5]
    Please do it step by step.
    Question: {question}
    """

    tools = [WolframAlpha(appid)]

    llm = genc.interop.langchain.CustomModel(
        uri="/cloud/gemini",
        config=genc.interop.gemini.create_config(api_key),
    )

    # Init langchain.agents.agent.Agent with components backed by C++ runtime.
    super().__init__(
        llm_chain=langchain.chains.LLMChain(llm=llm, prompt=PromptTemplate(
          input_variables=["query"],
          template=system_instruction)),
        allowed_tools=[tool.name for tool in tools],
        tools_list=tools,
        max_iterations=8,
    )


This is all! As you can see, this is a lot more compact, you get to use an API
you're familiar with, while at the same time being able to express the missing
bits in GenC's native authoring API, and convert the logic into a portable IR
that you can deploy in a variety of places (like what you've seen in the earlier
tutorials).

You can find all the code in an easy to run form in
[langchain_agent_demo.py](https://github.com/google/generative_computing/tree/master/generative_computing/python/examples/langchain_agent_demo.py).

## Compile & Run it

Here's how it all comes together. Make sure to fill in the missing `app_id` and
the `api_key` (as you did earlier).

In [ ]:
wolfram_app_id=""
gemini_api_key=""

agent = MathToolAgent(wolfram_app_id, gemini_api_key)

portable_ir = genc.interop.langchain.create_computation(agent)

# To run it localy
runner = genc.runtime.Runner(portable_ir,
                             genc.examples.executor.create_default_executor())
result = runner("""
  what is the result of (square root of 4) + 3 to the power of 2 + 3 *(8 +
  4) / 2 - 7"""
)

print(result)

## Save the IR to a file and deploy on Android

As in the preceding tutorials, the IR is, once again, portable, and can be
deployed on the phone. You can print and save it to a file as shown below.

In [ ]:
# Let's take a peak of how the IR or computation proto looks like
print(portable_ir)

with open("/tmp/genc_demo.pb", "wb") as f:
  f.write(portable_ir.SerializeToString())

After that, you can follow the same process as in
[Tutorial 1](https://github.com/google/generative_computing/tree/master/generative_computing/docs/tutorials/tutorial_1_simple_cascade.ipynb)
to load it in the “Generative Computing Demo” app on your Android phone, and
test it there.

For example, you can type a math query like the one below:

```What is the result of 6 * 9 + (square root of 4)```

This is the last tutorial currently included with GenC, but please stay tuned
for more coming in the future!